In [1]:
# import snowflake.connector
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine


import pandas as pd
import openai
# import json
# import PyPDF2


In [63]:
receivables_data = pd.read_excel("../files/LCIV - NT data.Receivables202203311037557.xlsx")
total_net_assets_data = pd.read_excel("../files/LCIV - NT data.Total Net Asset.202301111159408 (1).xlsx")
liabilities_data = pd.read_excel("../files/LCIV - NT data.Liabilities 202301111159183.xlsx")

In [64]:
receivables_data["Table type"] = "Receivables"
liabilities_data["Table type"] = "Liabilities"
total_net_assets_data["Table type"] = "Total net assets"

In [65]:
receivables_data["Date"] = liabilities_data["Date"][0]

In [66]:
receivables_data

,Fund,Date,Description,Start,Change,End,For More Information,Table type
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Income,1.100336e+06,1085218.41,2.185554e+06,See Accrued Gross Income,Receivables
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Withholding,-7.114840e+03,2118.51,-4.996330e+03,See Accrued Net Withholding,Receivables
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Spot Contracts,1.987708e+05,3307464.82,3.506236e+06,See Spot Contracts Payable/Receivable,Receivables
3,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Securities Sold,1.580125e+07,-7804936.51,7.996314e+06,See Securities Purchased Payable/Sold Receivable,Receivables
4,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Reclaims,3.265934e+06,65801.91,3.331735e+06,See Reclaim Receivable,Receivables
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Accrued Gross Income,4.839237e+06,-4839236.63,0.000000e+00,See Accrued Gross Income,Receivables
6,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Securities Sold,-4.839237e+06,4839236.63,0.000000e+00,See Securities Purchased Payable/Sold Receivable,Receivables
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Accrued Gross Income,2.118886e+06,-2118886.19,0.000000e+00,See Accrued Gross Income,Receivables
8,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Accrued Gross Income,0.000000e+00,10838935.29,1.083894e+07,See Accrued Gross Income,Receivables
9,1745443 - LCIV REAL RETURN FND,30 Dec 22,Accrued Gross Income,2.075014e+06,-2075013.77,0.000000e+00,See Accrued Gross Income,Receivables


In [77]:
joined_data = pd.concat([receivables_data, total_net_assets_data, liabilities_data], axis=0)

In [79]:
joined_data.head(15)

,Fund,Date,Description,Start,Change,End,For More Information,Table type
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Income,1100336.05,1085218.41,2185554.46,See Accrued Gross Income,Receivables
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Withholding,-7114.84,2118.51,-4996.33,See Accrued Net Withholding,Receivables
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Spot Contracts,198770.82,3307464.82,3506235.64,See Spot Contracts Payable/Receivable,Receivables
3,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Securities Sold,15801250.20,-7804936.51,7996313.69,See Securities Purchased Payable/Sold Receivable,Receivables
4,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Reclaims,3265933.54,65801.91,3331735.45,See Reclaim Receivable,Receivables
5,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Accrued Gross Income,4839236.63,-4839236.63,0.00,See Accrued Gross Income,Receivables
6,1736938 - LCIV DIVERSIFD GROWTH FND,30 Dec 22,Securities Sold,-4839236.63,4839236.63,0.00,See Securities Purchased Payable/Sold Receivable,Receivables
7,1740040 - LCIV GB TOTAL RETURN FND,30 Dec 22,Accrued Gross Income,2118886.19,-2118886.19,0.00,See Accrued Gross Income,Receivables
8,1740041 - LCIV ABSOLUTE RETURN FND,30 Dec 22,Accrued Gross Income,0.00,10838935.29,10838935.29,See Accrued Gross Income,Receivables
9,1745443 - LCIV REAL RETURN FND,30 Dec 22,Accrued Gross Income,2075013.77,-2075013.77,0.00,See Accrued Gross Income,Receivables


In [68]:
joined_data = joined_data.sort_values(by='Fund', ascending=True)

In [70]:
joined_data.to_csv("joined_data.csv", index=False)

In [82]:
grouped = joined_data.groupby(['Fund', 'Table type', 'Date']).agg({'End': 'sum', 'Start':'sum'}).reset_index()


In [83]:
grouped

,Fund,Table type,Date,End,Start
0,1736917 - LCIV GB ALPHA GROWTH FND,Liabilities,30 Dec 22,-3.318115e+06,-8.797284e+06
1,1736917 - LCIV GB ALPHA GROWTH FND,Receivables,30 Dec 22,1.888321e+07,2.208791e+07
2,1736917 - LCIV GB ALPHA GROWTH FND,Total net assets,30 Dec 22,8.900892e+08,1.229889e+09
3,1736938 - LCIV DIVERSIFD GROWTH FND,Liabilities,30 Dec 22,-7.503213e+05,-5.937032e+06
4,1736938 - LCIV DIVERSIFD GROWTH FND,Receivables,30 Dec 22,0.000000e+00,0.000000e+00
5,1736938 - LCIV DIVERSIFD GROWTH FND,Total net assets,30 Dec 22,7.863990e+08,7.727695e+08
6,1740040 - LCIV GB TOTAL RETURN FND,Liabilities,30 Dec 22,-2.472860e+05,-8.974092e+05
7,1740040 - LCIV GB TOTAL RETURN FND,Receivables,30 Dec 22,0.000000e+00,2.118886e+06
8,1740040 - LCIV GB TOTAL RETURN FND,Total net assets,30 Dec 22,2.150252e+08,2.117766e+08
9,1740041 - LCIV ABSOLUTE RETURN FND,Liabilities,30 Dec 22,-6.205894e+04,-3.629060e+05


In [84]:
text = "DF: {'Valuation Date': [datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30)], 'Account ID (FA)': ['1736917', '1736917', '1736917', '1736917', '1748761', '1748760', '1748760', '1748760', '1750023', '1751063'], 'Account ID (FA)/Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1748761 - LCIV UK EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751063 - LCIV EQTY INCOME FND'], 'Account ID (Custody)': ['LLGQ01', 'LLGQ01', 'LLGQ01', 'LLGQ01', 'LLMJ01', 'LLNW01', 'LLNW01', 'LLNW01', 'LLLV01', 'LLEP01'], 'Security Description (Short)': ['UNITED STATES DOLLAR', 'PRINCIPAL CASH', 'S A RANDS', 'SWEDISH KRONA', 'UNITED STATES DOLLAR', 'PRINCIPAL CASH', 'SWISS FRANC', 'SWEDISH KRONA', 'PRINCIPAL CASH', 'PRINCIPAL CASH'], 'Security Number (Full)': ['USD', 'CASH 00001', 'ZAR', 'SEK', 'USD', 'CASH 00001', 'CHF', 'SEK', 'CASH 00001', 'CASH 00001'], 'Issue Currency': ['USD', 'GBP', 'ZAR', 'SEK', 'USD', 'GBP', 'CHF', 'SEK', 'GBP', 'GBP'], 'Long/Short Indicator': ['L', 'L', 'S', 'L', 'S', 'L', 'L', 'S', 'S', 'L'], 'Country of Risk': ['US', 'GB', 'ZA', 'SE', 'US', 'GB', 'CH', 'SE', 'GB', 'GB']}"

In [91]:
text.split("DF: ")[1]

"{'Valuation Date': [datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30)], 'Account ID (FA)': ['1736917', '1736917', '1736917', '1736917', '1748761', '1748760', '1748760', '1748760', '1750023', '1751063'], 'Account ID (FA)/Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1748761 - LCIV UK EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751063 - LCIV EQTY INCOME FND'], 'Account ID (Custody)': ['LLGQ01', 'LLGQ01', 'LLGQ01', 'LLGQ01', 'LLMJ01', 'LLNW01', 'LLNW01', 'LLNW01', 'LLLV01', 'LLEP01'], 'Security Description (Short)': ['UNITED STATES DOLLAR', 'PRINCIPAL CASH', 

In [92]:
import ast
last_observation_data = ast.literal_eval(text.split("DF: ")[1])

ValueError: malformed node or string on line 1: <ast.Call object at 0x1181bbd90>

In [118]:
import ast
import datetime

def parse_datetime_dict(astr,debug=False):
    try: tree=ast.parse(astr)
    except SyntaxError: raise ValueError(astr)
    for node in ast.walk(tree):
        if isinstance(node,(ast.Module,ast.Expr,ast.Dict,ast.Str,
                            ast.Attribute,ast.Num,ast.Name,ast.Load, ast.Tuple)): continue
        if (isinstance(node,ast.Call)
                and isinstance(node.func, ast.Attribute)
                and node.func.attr == 'datetime'): continue
        if isinstance(node,ast.List): 
            print("ere")
            
        if debug:
            attrs=[attr for attr in dir(node) if not attr.startswith('__')]
            print(node)
            for attrname in attrs:
                print('    {k} ==> {v}'.format(k=attrname,v=getattr(node,attrname)))
        raise ValueError(astr)
    return eval(astr)

good_strings=["{'the_datetime': datetime.datetime(2010, 11, 21, 0, 56, 58)}"]
bad_strings=["__import__('os'); os.unlink",
             "import os; os.unlink",
             "import(os)", # SyntaxError
             ]

for astr in good_strings:
    result=parse_datetime_dict(astr)    
    print('{s} ... [PASSED]'.format(s=astr))


{'the_datetime': datetime.datetime(2010, 11, 21, 0, 56, 58)} ... [PASSED]


In [119]:
result

{'the_datetime': datetime.datetime(2010, 11, 21, 0, 56, 58)}

In [120]:
parse_datetime_dict(text.split("DF: ")[1])

ere


ValueError: {'Valuation Date': [datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30), datetime.date(2021, 9, 30)], 'Account ID (FA)': ['1736917', '1736917', '1736917', '1736917', '1748761', '1748760', '1748760', '1748760', '1750023', '1751063'], 'Account ID (FA)/Name': ['1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1736917 - LCIV GB ALPHA GROWTH FND', '1748761 - LCIV UK EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1748760 - LCIV GB EQTY FND', '1750023 - LCIV GB EQ FOCUS FND', '1751063 - LCIV EQTY INCOME FND'], 'Account ID (Custody)': ['LLGQ01', 'LLGQ01', 'LLGQ01', 'LLGQ01', 'LLMJ01', 'LLNW01', 'LLNW01', 'LLNW01', 'LLLV01', 'LLEP01'], 'Security Description (Short)': ['UNITED STATES DOLLAR', 'PRINCI

In [5]:
# for column in columns:
#     column_values = list(db[column].values)
#     new_column_values = []
#     for col_val in column_values:
#         new_col_val = col_val.replace(",", "")
#         new_column_values.append(float(new_col_val))
#     db[column] = new_column_values

In [7]:
# db.to_csv("balance_data.csv", index=False)


In [6]:
url = URL(
    user='itsmejamshid',
    password='Mucklab17',
    account='ln12766.uae-north.azure',
    warehouse='COMPUTE_WH',
    database='FREEZY_SAMPLE',
    schema='CYBERSYN_FINANCIAL',
    role = 'ACCOUNTADMIN'
)
engine = create_engine(url)

In [7]:
engine.connect()

In [8]:
table_inputs = [
    ("TESTFORADF.PUBLIC.BALANCE", ""),
    ("TESTFORADF.PUBLIC.CASH", ""),
    ("TESTFORADF.PUBLIC.INVESTMENT", ""),
    ("TESTFORADF.PUBLIC.TURNOVER", ""),
    
    
]

In [9]:
# QUALIFIED_TABLE_NAME = "TESTFORADF.PUBLIC.BALANCE"
# TABLE_DESCRIPTION = ""


In [10]:
full_context = ""
for table_input in table_inputs:
    table_name, table_description = table_input
    table_str_list = table_name.split(".")
    query = f"""
        SELECT COLUMN_NAME, DATA_TYPE FROM {table_str_list[0].upper()}.INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = '{table_str_list[1].upper()}' AND TABLE_NAME = '{table_str_list[2].upper()}'
        """
    connection = engine.connect()
    
    columns = pd.read_sql(query, connection)
    # connection = st.session_state.connection

    columns = pd.read_sql(query, connection)
    columns = "\n".join(
        [
            f"- **{columns['column_name'][i]}**: {columns['data_type'][i]}"
            for i in range(len(columns["column_name"]))
        ]
    )
    context = f"""
Here is the table name <tableName> {'.'.join(table_str_list)} </tableName>

<tableDescription>{table_description}</tableDescription>

Here are the columns of the {'.'.join(table_str_list)}

<columns>\n\n{columns}\n\n</columns>
"""
    full_context += context + '\n\n\n'

In [11]:
print(full_context)


Here is the table name <tableName> TESTFORADF.PUBLIC.BALANCE </tableName>

<tableDescription></tableDescription>

Here are the columns of the TESTFORADF.PUBLIC.BALANCE

<columns>

- **Total Net Assets**: FLOAT
- **Liabilities**: FLOAT
- **Receivables**: FLOAT
- **Date**: TEXT
- **Fund Name**: TEXT

</columns>




Here is the table name <tableName> TESTFORADF.PUBLIC.CASH </tableName>

<tableDescription></tableDescription>

Here are the columns of the TESTFORADF.PUBLIC.CASH

<columns>

- **Account ID (FA)/Name**: TEXT
- **Country of Risk**: TEXT
- **Issue Currency**: TEXT
- **Account ID (FA)**: TEXT
- **Security Description (Short)**: TEXT
- **Valuation Date**: DATE
- **Long/Short Indicator**: TEXT
- **Security Number (Full)**: TEXT
- **Account ID (Custody)**: TEXT

</columns>




Here is the table name <tableName> TESTFORADF.PUBLIC.INVESTMENT </tableName>

<tableDescription></tableDescription>

Here are the columns of the TESTFORADF.PUBLIC.INVESTMENT

<columns>

- **Account ID (FA)**: 

In [52]:
system = f"""
You will be acting as an AI Snowflake SQL Expert named Freezy.
Your goal is to give correct, executable sql query to users.
You are given one or multiple tables, the table name are in <tableName> tags, the columns are in <columns> tag.
If user asks you to produce output table, you need to produce SQL query to create or replace ouput table based on the user's query.
User will give you query and you need to extract necessary filters and the tables to aggregate for output table.
You have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
Since all of the columns in the tables are distinct and do not share common name, you have to use the UNION function in sql query to merge tables into single output table by appending them.
If there are multiple tables, since the columns in the tables are different, we will need to fill in the missing columns with NULL values to make the number of columns equal.
Before producing any sql query to create output table, you should ask what would be the name for the output table name from user.
When there are common column names among the tables, you need make them distinctive by merging table's name and column name to create new distinctive name.
Other than asking you produce output table, user might give you other queries. You should respond to it with giving correct, executable sql query to users.
You will be replying to users who will be confused if you don't respond in the character of Freezy.

{full_context}

Here are 6 critical rules for the interaction you must abide:
<rules>
1. You MUST MUST wrap the generated sql code within ``` sql code markdown in this format e.g
```sql
(select 1) union (select 2)
```
2. You HAVE to use the UNION function in sql query to create output table.
3. If I don't tell you to find a limited set of results in the sql query or question, you MUST limit the number of responses to 10.
4. Text / string where clauses must be fuzzy match e.g ilike %keyword%
5. Make sure to generate a single snowflake sql code, not multiple. 
6. You should only use the table columns given in <columns>, and the table given in <tableName>, you MUST NOT hallucinate about the table names
7. DO NOT put numerical at the very front of sql variable.
8. USE "ilike %keyword%" for match queries (especially for variable_name column)
</rules>


Wrap the generated sql code with ``` sql code markdown in this format e.g:
```sql
(select 1) union (select 2)
```

For each question from the user, make sure to include a query in your response.
"""
# print(system)

In [53]:
print(system)


You will be acting as an AI Snowflake SQL Expert named Freezy.
Your goal is to give correct, executable sql query to users.
You are given one or multiple tables, the table name are in <tableName> tags, the columns are in <columns> tag.
If user asks you to produce output table, you need to produce SQL query to create or replace ouput table based on the user's query.
User will give you query and you need to extract necessary filters and the tables to aggregate for output table.
You have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
When there is no data, it should be treated as null.
Since all of the columns in the tables are distinct and do not share common name, you have to use the UNION function in sql query to merge tables into single output table by appending them.
If there are multiple tables, since the columns in the tables are different, we will need to fill in the missing columns with NULL values to make the number of co

In [54]:
openai.api_key = "d3e70cf965e3499c868d5fd0448ecf99"
openai.api_base =  "https://ai-stage1.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
# openai.api_version = '2023-03-15-preview' # this may change in the future
openai.api_version = '2023-03-15-preview'

chatgpt = 'gpt-3.5-turbo'

In [18]:
prompt = f"""<|im_start|>system
{system}
<|im_end|>
<|im_start|>user
Question: Give me the output table from the all of the table I created under the name 'OUTPUT'.
<|im_end|>
<|im_start|>assistant
Answer:"""
response = openai.Completion.create(engine="chatgpt", prompt=prompt, temperature=0.5, max_tokens=1024)
print(response['choices'][0]['text'].strip())


Before I can provide you with the SQL query to create the output table, could you please confirm if you want me to include all columns from all tables? Also, do you want me to replace any existing 'OUTPUT' table or create a new one?<|im_end|>


In [100]:
content = r['choices'][0].message.content

In [101]:
import re
sql_match = re.search(r"```sql\n(.*)\n```", content, re.DOTALL)

In [103]:
new_content = content.replace(sql_match.group(1), '')

In [105]:
print(new_content)

Understood. To create the output table with all columns from all tables, you can use the following SQL query:

```sql

```

This will create a new table named 'OUTPUT' that contains all columns from all tables.


In [102]:
print(sql_match.group(1))

CREATE OR REPLACE TABLE OUTPUT AS
(
SELECT * FROM TESTFORADF.PUBLIC.BALANCE
UNION
SELECT * FROM TESTFORADF.PUBLIC.CASH
UNION
SELECT * FROM TESTFORADF.PUBLIC.INVESTMENT
UNION
SELECT * FROM TESTFORADF.PUBLIC.TURNOVER
);


# chat

In [86]:
messages = [
    {
        "role":"system",
        "content": system
    }
]

In [87]:
user_question = " Give me the output table from the all of the table I created under the name 'OUTPUT'."

In [92]:
user_question = "all of them"

In [93]:
messages.append({
    "role": "user",
    "content": user_question
})

In [94]:
r = openai.ChatCompletion.create(
    engine="chatgpt",
    messages=messages,
    temperature=0.1,
)

In [95]:
r['choices'][0].message.content

"Understood. To create the output table with all columns from all tables, you can use the following SQL query:\n\n```sql\nCREATE OR REPLACE TABLE OUTPUT AS\n(\nSELECT * FROM TESTFORADF.PUBLIC.BALANCE\nUNION\nSELECT * FROM TESTFORADF.PUBLIC.CASH\nUNION\nSELECT * FROM TESTFORADF.PUBLIC.INVESTMENT\nUNION\nSELECT * FROM TESTFORADF.PUBLIC.TURNOVER\n);\n```\n\nThis will create a new table named 'OUTPUT' that contains all columns from all tables."

In [91]:
messages.append(
    {
        "role":r['choices'][0].message.role,
        "content":r['choices'][0].message.content
    
    }
)

In [34]:
user_question = "all of the columns"
messages.append({
    "role": "user",
    "content": user_question
})

In [ ]:
r = openai.ChatCompletion.create(
    engine="chatgpt",
    messages=messages,
    temperature=0.1,
)

# doing with gpt-4

In [40]:
openai.api_key = "cdf63ece486a426da741e31ef181950c"
openai.api_base =  "https://ai-stage2.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
# openai.api_version = '2023-03-15-preview' # this may change in the future
openai.api_version = '2023-08-01-preview'

In [41]:
messages = [
    {
        "role":"system",
        "content": system
    }
]

In [42]:
user_question = "Give me the output table from the all of the table I created under the name 'OUTPUT'"

In [47]:
messages.append({
    "role": "user",
    "content": user_question
})

In [48]:
r = openai.ChatCompletion.create(
    engine="gpt-4",
    messages=messages,
    temperature=0.1,
)

In [50]:
print(r['choices'][0].message.content)

, I can help with that. Let's create a new table named 'OUTPUT' that combines all the columns from the tables you provided. Here's the SQL query:

```sql
CREATE OR REPLACE TABLE OUTPUT AS
(
SELECT 
    "Total Net Assets",
    "Liabilities",
    "Receivables",
    "Date",
    "Fund Name",
    NULL AS "Account ID (FA)/Name",
    NULL AS "Country of Risk",
    NULL AS "Issue Currency",
    NULL AS "Account ID (FA)",
    NULL AS "Security Description (Short)",
    NULL AS "Valuation Date",
    NULL AS "Long/Short Indicator",
    NULL AS "Security Number (Full)",
    NULL AS "Account ID (Custody)",
    NULL AS "Primary Exchange Code",
    NULL AS "Account Name (Short)",
    NULL AS "Security Category Industry Description",
    NULL AS "Market Value Traded (Base)",
    NULL AS "ISIN",
    NULL AS "Issuer Name",
    NULL AS "Account Number",
    NULL AS "N-CATG-NAME",
    NULL AS "Recognition date",
    NULL AS "A-BAS-BSE",
    NULL AS "N-INV-SUB-CATG",
    NULL AS "Through date"
FROM TESTFOR

In [82]:
messages.append(
    {
        "role":r['choices'][0].message.role,
        "content":r['choices'][0].message.content
     
    }
)

In [83]:
messages.append(
    {
        "role":"user",
        "content":"The sql you gave me again produce this error: invalid number of result columns for set operator input branches, expected 5, got 9 in branch 2"
    
    }
)

In [64]:
r = openai.ChatCompletion.create(
    engine="gpt-4",
    messages=messages,
    temperature=0.1,
)

In [99]:
response_content = r['choices'][0].message.content

In [108]:
print(response_content)

Sure thing! Here's the SQL query you need. It will create an output table from the BALANCE and CASH tables, filtering for Receivables greater than 0.

```sql
CREATE OR REPLACE TABLE TESTFORADF.PUBLIC.LCIV_OUTPUT AS
(
    SELECT 
        "Total Net Assets",
        "Receivables",
        "Liabilities",
        "Date",
        "Fund Name",
        NULL AS "Country of Risk",
        NULL AS "Account ID (FA)/Name",
        NULL AS "Issue Currency",
        NULL AS "Account ID (FA)",
        NULL AS "Security Description (Short)",
        NULL AS "Valuation Date",
        NULL AS "Security Number (Full)",
        NULL AS "Long/Short Indicator",
        NULL AS "Account ID (Custody)"
    FROM TESTFORADF.PUBLIC.BALANCE
    WHERE "Receivables" > 0

    UNION

    SELECT 
        NULL AS "Total Net Assets",
        NULL AS "Receivables",
        NULL AS "Liabilities",
        NULL AS "Date",
        NULL AS "Fund Name",
        "Country of Risk",
        "Account ID (FA)/Name",
        "Issue C

In [101]:
sql_match = re.search(r"```sql\n(.*)\n```", response_content, re.DOTALL)

In [102]:
print(sql_match.group(1))

CREATE OR REPLACE TABLE TESTFORADF.PUBLIC.LCIV_OUTPUT AS
(
    SELECT 
        "Total Net Assets",
        "Receivables",
        "Liabilities",
        "Date",
        "Fund Name",
        NULL AS "Country of Risk",
        NULL AS "Account ID (FA)/Name",
        NULL AS "Issue Currency",
        NULL AS "Account ID (FA)",
        NULL AS "Security Description (Short)",
        NULL AS "Valuation Date",
        NULL AS "Security Number (Full)",
        NULL AS "Long/Short Indicator",
        NULL AS "Account ID (Custody)"
    FROM TESTFORADF.PUBLIC.BALANCE
    WHERE "Receivables" > 0

    UNION

    SELECT 
        NULL AS "Total Net Assets",
        NULL AS "Receivables",
        NULL AS "Liabilities",
        NULL AS "Date",
        NULL AS "Fund Name",
        "Country of Risk",
        "Account ID (FA)/Name",
        "Issue Currency",
        "Account ID (FA)",
        "Security Description (Short)",
        "Valuation Date",
        "Security Number (Full)",
        "Long/Short I

SELECT balance."Total Net Assets", balance."Liabilities", balance."Receivables", balance."Date", balance."Fund Name", 
       cash."Country of Risk", cash."Account ID (FA)/Name", cash."Issue Currency", cash."Account ID (FA)", cash."Security Description (Short)", 
       cash."Valuation Date", cash."Security Number (Full)", cash."Long/Short Indicator", cash."Account ID (Custody)"
FROM TESTFORADF.PUBLIC.BALANCE as balance 
JOIN TESTFORADF.PUBLIC.CASH as cash 
ON balance."Date"=cash."Valuation Date" 
WHERE balance."Receivables" > 0
UNION ALL
SELECT balance."Total Net Assets", balance."Liabilities", balance."Receivables", balance."Date", balance."Fund Name", 
       cash."Country of Risk", cash."Account ID (FA)/Name", cash."Issue Currency", cash."Account ID (FA)", cash."Security Description (Short)", 
       cash."Valuation Date", cash."Security Number (Full)", cash."Long/Short Indicator", cash."Account ID (Custody)"
FROM TESTFORADF.PUBLIC.BALANCE as balance 
RIGHT JOIN TESTFORADF.PUBLIC.C

In [150]:
# response = openai.ChatCompletion.create(engine="gpt-4", prompt=prompt, temperature=0.5, max_tokens=1024)


SELECT balance."Total Net Assets", balance."Liabilities", balance."Receivables", balance."Date", balance."Fund Name", cash."Country of Risk", cash."Account ID (FA)/Name", cash."Issue Currency", cash."Account ID (FA)", cash."Security Description (Short)", cash."Valuation Date", cash."Security Number (Full)", cash."Long/Short Indicator", cash."Account ID (Custody)"
FROM TESTFORADF.PUBLIC.BALANCE as balance
JOIN TESTFORADF.PUBLIC.CASH as cash
ON balance."Fund Name" = cash."Account ID (FA)/Name"
WHERE balance."Receivables" > 0
UNION ALL
SELECT balance."Total Net Assets", balance."Liabilities", balance."Receivables", balance."Date", balance."Fund Name", cash."Country of Risk", cash."Account ID (FA)/Name", cash."Issue Currency", cash."Account ID (FA)", cash."Security Description (Short)", cash."Valuation Date", cash."Security Number (Full)", cash."Long/Short Indicator", cash."Account ID (Custody)"
FROM TESTFORADF.PUBLIC.BALANCE as balance
RIGHT JOIN TESTFORADF.PUBLIC.CASH as cash
ON balance.

In [119]:
if sql_match:
    sql = sql_match.group(1)
    # conn = st.experimental_connection("snowpark")
    results = pd.read_sql(sql, connection)
    print(results)

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 000904 (42000): SQL compilation error: error line 4 at position 4
invalid identifier 'BALANCE.LIABILITIES'
[SQL: CREATE OR REPLACE TABLE TESTFORADF.PUBLIC.LCIV_OUTPUT AS
SELECT 
    BALANCE."Total Net Assets" AS BALANCE_Total_Net_Assets,
    BALANCE.Liabilities AS BALANCE_Liabilities,
    BALANCE.Receivables AS BALANCE_Receivables,
    BALANCE.Date AS BALANCE_Date,
    BALANCE."Fund Name" AS BALANCE_Fund_Name,
    CASH."Country of Risk" AS CASH_Country_of_Risk,
    CASH."Account ID (FA)/Name" AS CASH_Account_ID_FA_Name,
    CASH."Issue Currency" AS CASH_Issue_Currency,
    CASH."Account ID (FA)" AS CASH_Account_ID_FA,
    CASH."Security Description (Short)" AS CASH_Security_Description_Short,
    CASH."Valuation Date" AS CASH_Valuation_Date,
    CASH."Security Number (Full)" AS CASH_Security_Number_Full,
    CASH."Long/Short Indicator" AS CASH_Long_Short_Indicator,
    CASH."Account ID (Custody)" AS CASH_Account_ID_Custody

In [2]:
with open("test_data/text.txt", "r") as f:
    whole_text = f.read()

In [9]:
S = 0
for line in whole_text.split("\n"):
    line_number = ''
    for x in line:
        if x.isnumeric() and len(line_number) != 2:
            line_number += x
    S += int(line_number)

In [10]:
S

35609

In [12]:
import pandas as pd

In [13]:
data = pd.read_csv("joined_data.csv")

In [15]:
data_csv = data.to_csv("asd.csv")

In [18]:
with open('asd.csv', 'r') as do:
    print(do.read())

,Fund,Date,Description,Start,Change,End,For More Information,Table type
0,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Gross Income,1100336.05,1085218.41,2185554.46,See Accrued Gross Income,Receivables
1,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Payables,,,,,Liabilities
2,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Expenses,-3886694.17,941513.01,-2945181.16,See Accrued Expenses,Liabilities
3,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Accrued Capital Expenses,-290231.92,-82568.02,-372799.94,,Liabilities
4,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Spot Contracts,-36362.67,36362.67,0.0,See Spot Contracts Payable/Receivable,Liabilities
5,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Income,-4583995.57,4583995.57,0.0,See Income Payable,Liabilities
6,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Unrealized Loss on Forward Contracts,0.0,-134.14,-134.14,See Unrealized G/L on Currency Contracts,Liabilities
7,1736917 - LCIV GB ALPHA GROWTH FND,30 Dec 22,Total Net Assets,12